In [16]:
import numpy as np
import pandas as pd
import os

In [17]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)                
    return allFiles

Объединяем все фотометрические данные в один dataframe

In [18]:
phot_gals = getListOfFiles('./phot_sdss_grps')
df = pd.DataFrame()
for gal in phot_gals:
    t = pd.read_csv(gal)
    t['iGrID']=gal[:-4].split('_')[-1]
    df = pd.concat([t, df]).reset_index(drop=True)

Добавляем новую колонку с цветом галактик

In [19]:
df['|dered_g-dered_r|']=np.abs(df['dered_g']-df['dered_r'])

Загрузим таблицу, с данными с каталога SDSS с группами, у которых количество галактик > 70

In [20]:
g_list_s = pd.read_csv('./glist_s_70.csv')
g_list_s.rename(columns={g_list_s.columns[3]:'objid'}, inplace=True)

Сделаем таблицу с галактиками, у которых есть и фотометрические данные и красные смещения

In [21]:
g_list = pd.merge(g_list_s, df, on='objid', how='left',suffixes=('_s','_photo'))
g_l = g_list[pd.notnull(g_list['|dered_g-dered_r|'])]

In [27]:
list_z_photo = g_l[['objid', 'ra', 'dec','z_gal','|dered_g-dered_r|','iGrID_photo']]
list_z_photo.rename(columns={list_z_photo.columns[5]:'iGrID'}, inplace=True)

In [28]:
list_z_photo.head()

,objid,ra,dec,z_gal,|dered_g-dered_r|,iGrID
0,1237650370488762520,195.544557,-2.495210,0.087696,0.88330,211
9,1237650370488828045,195.704581,-2.510970,0.086480,0.89793,211
12,1237650370488762557,195.589523,-2.501693,0.080822,0.88170,211
13,1237650370488762524,195.547180,-2.526123,0.085518,0.59039,211
15,1237650370488828042,195.720356,-2.520542,0.082964,0.91098,211
